In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas
import json
import os


URL = "https://www.sony.com.au/lenses/gallery"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# with open('source/sony-lenses.xml', 'w') as f:
#     f.write(soup.prettify())
#     f.close